# Projet VaR
# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
file=pd.ExcelFile('projetVaR_light_groupe1.xlsm')

In [3]:
data=file.parse('cleandata',header=None)

# Data Cleaning

In [4]:
spot=data[5:]
columns=data.iloc[4]
spot.columns=columns
spot=spot.set_index(spot.columns[0])
spot.index.name='Date'

In [5]:
stock=spot.iloc[:,:-10]

In [6]:
control_room=file.parse('control_room',header=None)

In [7]:
halton=file.parse('halton',header=None)

In [8]:
# references de données du portefeuille
portfolio=control_room.iloc[5:31,2:13].reset_index()
portfolio.columns=portfolio.iloc[0]
portfolio=portfolio.drop(columns=5,index=[0])

In [9]:
# Creation des cours de change
FX=spot.iloc[:,-10:]
FX['EUREUR']=1

In [10]:
#Creation d'un dictionnaire de devise
devise={}
devise["EUR"]='EUREUR'
for col_index in range(0,len(FX.columns),2):
    devise_etrangere=FX.columns[col_index][:3]
    devise[devise_etrangere]=FX.columns[col_index]

## Composition portefeuille

In [11]:
#Creation d'un dictionnaire qui recense les devises de cotations de chaques titres et d'un dictionnaire qui recence le poids des ces titres dans le portefeuille
devise_ptf=dict(zip(portfolio['code'],portfolio['dev']))
positions=dict(zip(portfolio['code'],portfolio['poids']))

In [12]:
#Conversion des valeurs du dictionnaire de change du portefeuille dans le dictionnaire de devise , ex : GBp=> GBpEUR
for asset in devise_ptf.keys():
    devise_ptf[asset]=devise[devise_ptf[asset]]


In [13]:
# Creation d'un DataFrame qui reprends les cours en EUROS des actifs du portefeuille
ptf_data=pd.DataFrame()

for asset in devise_ptf.keys():
    
    if asset not in FX.columns:
        if devise_ptf[asset]=='EUREUR':
            ptf_data[asset]=spot[asset]

        elif devise_ptf[asset]=='GBpEUR':
            ptf_data[asset]=spot[asset]*FX['GBpEUR']/100

        else:
            ptf_data[asset]=spot[asset]*FX[devise_ptf[asset]]
    
    else:
        
        ptf_data[asset]=FX[devise_ptf[asset]]


In [14]:
# Creation d'un Array numpy pour stocker les poids du portefeuille (pour éviter aussi de se reordonner par rapport aux colonns de "ptf_data")
poids_ptf=[]
for col in ptf_data.columns:
    poids_ptf.append(positions[col])
poids_ptf=np.array(poids_ptf)

In [15]:
ptf_data

,ASML NA,SAP GY,NESN SW,MC FP,URW NA,HSBA LN,FP FP,NOVN SW,BP/ LN,LIN GY,...,BNP FP,ALV GY,GLE FP,RIO LN,CGG FP,EUREUR,CHFEUR,GBpEUR,DKKEUR,SX5E
Date,,,,,,,,,,,,,,,,,,,,,
2020-09-28,314.8,134.66,101.891724,405.9,31.57,3.394359,28.8,74.916567,2.579184,204.1,...,31.82,166.84,11.452,51.468179,0.577,1,0.9273,1.1001,0.1343,3223.19
2020-09-29,318.25,134.16,101.57123,409.95,29.87,3.26947,28.33,74.53495,2.511523,202.3,...,30.995,164.88,11.028,51.610536,0.5592,1,0.9259,1.0953,0.1343,3214.3
2020-09-30,314.7,132.76,101.30351,399.4,31.5,3.323736,29.2,74.036615,2.482605,201.9,...,30.975,163.62,11.318,51.322232,0.5738,1,0.9265,1.1024,0.1343,3193.61
2020-10-01,321.4,133.74,102.088766,409.7,30.3,3.341767,28.49,74.52276,2.393872,201.4,...,30.655,162.18,11.048,50.998694,0.565,1,0.9269,1.0971,0.1344,3194.09
2020-10-02,319.9,132.76,102.28518,407.65,29.88,3.403144,28.21,74.52153,2.367405,199.65,...,30.49,162.8,11.038,51.831148,0.5622,1,0.927,1.1042,0.1343,3190.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-29,516.5,104.3,95.203056,554.9,66.52,4.890934,39.745,74.537424,3.515249,237,...,50.67,214.6,21.705,64.10504,1.0565,1,0.9048,1.1698,0.1345,3882.87
2021-03-30,516.3,103.56,95.971046,573.5,69.22,5.050663,40.25,74.156974,3.535992,238.5,...,52.62,219.05,22.565,65.418784,1.0475,1,0.9059,1.1728,0.1345,3926.2
2021-03-31,517,104.42,95.164156,568.1,68.34,4.9726,39.775,72.967618,3.462137,238.9,...,51.88,217.05,22.315,65.2125,1.0255,1,0.9034,1.175,0.1344,3919.21


In [16]:
#Calcul des log-rendements du ptf
rendements_ptf=np.log(1+ptf_data.pct_change())

# NAV

In [17]:
# Calcul de la perf du ptf en faisant la somme pondérée des performance
VL=pd.DataFrame()
VL['performance']=np.sum(poids_ptf*rendements_ptf,axis=1)

# Indice

In [18]:
indice=pd.DataFrame()
indice['SX5P']=spot['SX5P']

In [19]:
indice['performance']=spot.iloc[:,213]

In [20]:
indice_composition=control_room.iloc[6:56,14:20]
indice_composition.columns=control_room.iloc[5,14:20]

In [21]:
#Dictionnaire recensant le poids d'un actif dans l'indice (en proportion)
poids_indice=dict(zip(indice_composition['code'],indice_composition['poids']/indice_composition['poids'].sum()))

In [22]:
#Conversion des valeurs du dictionnaire de change du portefeuille dans le dictionnaire de devise , ex : GBp=> GBpEUR

devise_indice=dict(zip(indice_composition['code'],indice_composition['dev']))
for asset in devise_indice.keys():
    devise_indice[asset]=devise[devise_indice[asset]]

In [23]:
#Creation d'un Dataframe qui reprend le cours de chaque actions de l'indice en euros
indice_composed=pd.DataFrame()
for stock in poids_indice.keys():
    if devise_indice[stock]=='EUREUR':
        indice_composed[stock]=spot[stock]
    elif devise_indice[stock]=='GBpEUR':
        indice_composed[stock]=spot[stock]*FX[devise_indice[stock]]/100
    else:
        indice_composed[stock]=spot[stock]*FX[devise_indice[stock]]

In [24]:
#Log rendement des cours du portefeuille de l'indice

indice_reconstruit_perf=np.log(1+indice_composed.pct_change())

In [25]:
# Creation d'un numpy array qui reprends les poids des actifs de l'indice en pourcentage
list_weights_indice=[]

for col in indice_composed.columns:
    list_weights_indice.append(poids_indice[col])
    
weights_indice=np.array(list_weights_indice)

In [26]:
perf_indice_compose=pd.DataFrame()
perf_indice_compose['performance']=np.sum(weights_indice*indice_reconstruit_perf,axis=1)

# Fonctions

In [27]:
# Variance d'un ptf
def variance(poids,matrice_cov):
    return np.dot(poids.T,np.dot(matrice_cov,poids))

In [28]:
# Variance marginale d'une ligne dans le ptf
def marginal_variance(weights,volatilité,covariances,index):
    
    VEC_VAR=weights[index]**2*volatilité[index]**2
    COVARS=0
    var=0
    
    for i in range(index+1,len(covariances)):
        
        COVARS+=covariances[i,index]*weights[index]*weights[i]
          
    var=COVARS*2 + VEC_VAR
    
    return var

### Monte Carlo 

In [29]:
# Fonction qui fait une simulation de Monte Carlo
def Monte_Carlo(stocks,facteurs,randoms,t):
    # Fonction qui dépend d'un dictionnaire qui associe actions -> devise, des facteurs de vols/drifts, des epsilons et de l'horizon temporel
    Monte_Carlo=pd.DataFrame()
    
    
    for asset in stocks.keys():
    # Pour chaque actif qui n'est pas une devise    
        if asset not in FX.columns:
            
            # Si l'actif est coté en EUROS
            if facteurs.loc[asset]['dev']=='EUR':
                Monte_Carlo[asset]=facteurs.loc[asset]['spot']*facteurs.loc[asset]['drift']*np.exp((randoms[asset]*facteurs.loc[asset]['vol']*np.sqrt(t)).astype(float))

            else:
            # Sinon conversion, prise en compte de l'effet quanto
                devise_asset=stocks[asset]
                asset_effect=facteurs.loc[asset]['spot']*facteurs.loc[asset]['drift']*np.exp((randoms[asset]*facteurs.loc[asset]['vol']*np.sqrt(t)).astype(float))
                Monte_Carlo[asset]=asset_effect*facteurs.loc[asset]['q adj']


        else:
            #Si l'actif en question est une devise
            if asset!='EUREUR':
                Monte_Carlo[asset]=facteurs.loc[asset]['spot']*facteurs.loc[asset]['drift']*np.exp(randoms[asset]*facteurs.loc[asset]['vol']*np.sqrt(t))

            else:
                Monte_Carlo[asset]=1
    
    return Monte_Carlo

In [30]:
def performance(monte_carlo,facteurs):
    
    # Calcule le log-rendement du Monte Carlo par rapport au spot
    performance=pd.DataFrame()
    for columns in monte_carlo.columns:
        if columns!='EUREUR':
            performance[columns]=monte_carlo[columns]/facteurs.loc[columns]['spot']
            
        else:
            performance[columns]=monte_carlo[columns]
            
    performance=np.log(performance)
    
    return performance

In [31]:
def portefeuille(poids_dict,perf_simulation):
    
    # Permet de faire le produit des poids par le vecteurs de performance, permet de reajuster l'ordre des poids par rapport aux colonne de la performance des actifs
    weights=[]
    for col in perf_simulation:
        weights.append(poids_dict[col])
    weights=np.array(weights)
    performance_portefeuille=pd.DataFrame()
    performance_portefeuille['performance']=np.sum(weights*perf_simulation,axis=1)
    
    return performance_portefeuille

### Calcul de VaR

In [32]:
def VaR_CVar(perf):
    #Renvoie la VaR et CVaR
    VaR=np.percentile(perf,5)
    CVaR=perf[perf<VaR].mean()

        
    return VaR,CVaR
    

# Question 1 : Expositions aux actions

In [33]:
# Exposition =(Montant des actions + futures) / total bilan
actions=portfolio[portfolio['type']!='liquidités']['montant eur'].sum()

In [34]:
bilan=portfolio[portfolio['type']!='future']['montant eur'].sum()

In [35]:
exposition=actions/bilan*100
exposition

97.91346219970326

# Question 2 : VaR historique

## a)

In [36]:
VaR_portfolio=np.percentile(VL['performance'],5)
VaR_indice=np.percentile(perf_indice_compose,5)

In [37]:
print("VaR du portefeuille est de " + str(round(VaR_portfolio,4)*100)+'%'," ," ,"VaR de l'indice est de " + str(round(VaR_indice,4)*100)+'%')

VaR du portefeuille est de -1.54%  , VaR de l'indice est de -1.3%


## b)

In [38]:
VaR_relative=np.percentile(VL['performance']-perf_indice_compose['performance'],5)
print(str(round(VaR_relative,4)*100)+'%')

-0.72%


## c)

In [39]:
VaR_relative_taux=(1-VaR_relative/VaR_portfolio)
print(str(round(VaR_relative_taux*100,2))+'%')

53.06%


# Question 3 : VaR parametrique

## a) Indice

In [40]:
vol_indice=np.sqrt(variance(weights_indice,np.log(1+indice_composed.pct_change()).cov())*252)
vol_historique_indice=perf_indice_compose['performance'].std()*np.sqrt(252)
print ( vol_indice,',',vol_historique_indice)

0.15439370641818298 , 0.1537984541306641


## b) Portefeuille

In [41]:
volatilité=np.sqrt(variance(poids_ptf,rendements_ptf.cov())*252)
volatilité_vl=VL['performance'].std()*np.sqrt(252)
print(volatilité,',',volatilité_vl)

0.20773212013916303 , 0.2069391534358266


# Question 4 : VaR Monte Carlo

In [42]:
facteurs=control_room.iloc[6:119,22:34]
facteurs.columns=control_room.iloc[5,22:34]

In [43]:
facteurs=facteurs.set_index('code')

In [44]:
epsilons=halton.iloc[:,114:]
epsilons=epsilons.iloc[1:,:]

In [45]:
epsilons.columns=facteurs.index

In [46]:
haltons=halton.iloc[1:,:113]
haltons.columns=facteurs.index

In [47]:
t=20/250

## a)

In [48]:
monte_carlo=Monte_Carlo(devise_ptf,facteurs,epsilons,t)
monte_carlo

,ASML NA,SAP GY,NESN SW,MC FP,URW NA,HSBA LN,FP FP,NOVN SW,BP/ LN,LIN GY,...,BNP FP,ALV GY,GLE FP,RIO LN,CGG FP,EUREUR,CHFEUR,GBpEUR,DKKEUR,SX5E
1,509.035274,111.430174,101.992548,574.314544,71.014434,4.275393,38.656658,75.509227,3.402859,247.293253,...,56.952327,222.075988,24.759710,69.047370,0.942752,1,0.899820,1.152935,0.134928,3968.839959
2,477.176931,105.230572,107.864647,541.494813,62.061915,3.409012,34.927909,76.381677,2.840711,230.476778,...,41.260542,198.664916,17.773766,47.222293,0.865952,1,0.889816,1.154835,0.134453,3660.287898
3,562.812061,105.647895,114.327983,598.619152,51.277377,4.611625,32.738357,85.691605,2.418412,261.300748,...,45.938621,214.393405,19.514184,63.194429,0.830249,1,0.902857,1.104783,0.134699,3964.068542
4,517.902730,124.700847,101.194357,552.799855,51.962892,4.285928,33.346100,76.623357,2.736936,222.801986,...,42.935025,198.917468,18.101868,63.996345,0.979667,1,0.912272,1.124582,0.133875,3717.028492
5,453.551988,93.817374,109.842407,611.085816,85.709853,4.445495,49.485638,80.344707,4.293484,240.976963,...,61.020804,236.999839,26.762569,58.482270,1.726050,1,0.908131,1.150164,0.134266,4073.275858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,507.524605,96.240276,102.917070,553.608537,78.907373,4.169939,38.406756,78.562460,3.093078,232.800400,...,45.281849,206.728254,21.223055,66.076337,0.987257,1,0.900611,1.156529,0.134558,3715.399330
9997,549.013167,108.479573,103.813093,593.013797,52.131768,4.340053,34.929345,70.026566,2.989638,229.777066,...,46.589633,208.959600,21.324600,62.110193,1.088458,1,0.920073,1.102876,0.134580,3750.766197
9998,511.771892,105.251397,106.749289,588.792577,85.668127,4.616406,40.978609,80.655690,3.305928,223.201933,...,54.494126,214.649493,23.572524,54.949836,1.069126,1,0.909238,1.167098,0.134379,3929.624314
9999,600.112070,127.448504,114.050991,592.767811,58.349772,4.668955,38.044586,83.783543,2.953615,253.520515,...,50.177905,227.469004,22.811501,53.935510,1.128979,1,0.914431,1.121917,0.134591,4230.520457


## b)

## VaR Portefeuille

In [49]:
perf=performance(monte_carlo,facteurs)

In [50]:
portefeuille_monte_carlo=portefeuille(positions,perf)

In [51]:
VaR_CVar(portefeuille_monte_carlo)

(-0.10370683924800836,
 performance   -0.128756
 dtype: float64)

In [52]:
print("VaR du portefeuille est de " +str(round(VaR_CVar(portefeuille_monte_carlo)[0]*100,3)) + "%")
print("CVaR du portefeuille est de " +str(round(VaR_CVar(portefeuille_monte_carlo)[1][0]*100,3)) + "%")

VaR du portefeuille est de -10.371%
CVaR du portefeuille est de -12.876%


### VaR indice

In [53]:
monte_carlo_indice=Monte_Carlo(devise_indice,facteurs,epsilons,t)

In [54]:
perf_monte_carlo_actions=performance(monte_carlo_indice,facteurs)

In [55]:
perf_indice_monte_carlo=portefeuille(poids_indice,perf_monte_carlo_actions)
VaR_CVar(perf_indice_monte_carlo)

(-0.07804323238887537,
 performance   -0.096633
 dtype: float64)

In [56]:
print("VaR de l'indice est de " +str(round(VaR_CVar(perf_indice_monte_carlo)[0]*100,3)) + "%")
print("CVaR de l'indice est de " +str(round(VaR_CVar(perf_indice_monte_carlo)[1][0]*100,3)) + "%")

VaR de l'indice est de -7.804%
CVaR de l'indice est de -9.663%


### VaR relative et CVaR

In [57]:
performance_relative=pd.DataFrame()
performance_relative['performance']=portefeuille_monte_carlo['performance']-perf_indice_monte_carlo['performance']
VaR_CVar(performance_relative)

(-0.04956580308799596,
 performance   -0.060617
 dtype: float64)

In [58]:
print("VaR relative est de " +str(round(VaR_CVar(performance_relative)[0]*100,3)) + "%")
print("CVaR relative est de " +str(round(VaR_CVar(performance_relative)[1][0]*100,3)) + "%")

VaR relative est de -4.957%
CVaR relative est de -6.062%


## Question 5) Stress Tests

## a)

In [59]:
# Copie des facteurs non stressé
stress_test=facteurs.copy()

# Multiplication de la vol par 1.5
stress_test['vol']=stress_test['vol']*1.5

In [60]:
# Ajustement de la corrélation, drift du au changement de volatilité
for i in range(len(stress_test)):
    if stress_test.iloc[i,1]!='EUR':
        stress_test.iloc[i,6]=np.exp(-0.5*stress_test.iloc[i,3]*stress_test.iloc[i,4]**2*t)
        
    else: 
        stress_test.iloc[i,6]=1

### VaR portefeuille stressé

In [61]:
monte_carlo_stress=Monte_Carlo(devise_ptf,stress_test,epsilons,t)

In [62]:
perf=performance(monte_carlo_stress,stress_test)
portefeuille_monte_carlo=portefeuille(positions,perf)

In [63]:
VaR_CVar(portefeuille_monte_carlo)

(-0.23594993605423473,
 performance   -0.273523
 dtype: float64)

In [64]:
print("VaR portefeuille stressé est de " +str(round(VaR_CVar(portefeuille_monte_carlo)[0]*100,3)) + "%")
print("CVaR portefeuille stressé est de " +str(round(VaR_CVar(portefeuille_monte_carlo)[1][0]*100,3)) + "%")

VaR portefeuille stressé est de -23.595%
CVaR portefeuille stressé est de -27.352%


### VaR indice stressé

In [65]:
monte_carlo_indice=Monte_Carlo(devise_indice,stress_test,epsilons,t)
perf_monte_carlo_actions=performance(monte_carlo_indice,stress_test)
perf_indice_monte_carlo=portefeuille(poids_indice,perf_monte_carlo_actions)

In [66]:
VaR_CVar(perf_indice_monte_carlo)

(-0.3262065795235325,
 performance   -0.354091
 dtype: float64)

In [67]:
print("VaR de l'indice stressé est de " +str(round(VaR_CVar(perf_indice_monte_carlo)[0]*100,3)) + "%")
print("CVaR de l'indice stressé est de " +str(round(VaR_CVar(perf_indice_monte_carlo)[1][0]*100,3)) + "%")

VaR de l'indice stressé est de -32.621%
CVaR de l'indice stressé est de -35.409%


### VaR relative et CVaR stressée

In [68]:
perf_relative_stress=pd.DataFrame()
perf_relative_stress['performance']=portefeuille_monte_carlo['performance']-perf_indice_monte_carlo['performance']

VaR_CVar(perf_relative_stress)

(0.0544033491260033,
 performance    0.037827
 dtype: float64)

In [69]:
print("VaR relative stressé est de " +str(round(VaR_CVar(perf_relative_stress)[0]*100,3)) + "%")
print("CVaR relative stressé est de " +str(round(VaR_CVar(perf_relative_stress)[1][0]*100,3)) + "%")

VaR relative stressé est de 5.44%
CVaR relative stressé est de 3.783%


## b)

## Vols initiales

### Portefeuille

In [70]:
monte_carlo=Monte_Carlo(devise_ptf,facteurs,haltons,t)
perf_monte_carlo=performance(monte_carlo,facteurs)
portefeuille_monte_carlo=portefeuille(positions,perf_monte_carlo)

In [71]:
VaR_CVar(portefeuille_monte_carlo)

(-0.04711140151769446,
 performance   -0.057528
 dtype: float64)

In [72]:
print("VaR portefeuille est de " +str(round(VaR_CVar(portefeuille_monte_carlo)[0]*100,3)) + "%")
print("CVaR du portefeuille est de " +str(round(VaR_CVar(portefeuille_monte_carlo)[1][0]*100,3)) + "%")

VaR portefeuille est de -4.711%
CVaR du portefeuille est de -5.753%


### Indice

In [73]:
monte_carlo_indice=Monte_Carlo(devise_indice,facteurs,haltons,t)
perf_monte_carlo_actions=performance(monte_carlo_indice,facteurs)
perf_indice_monte_carlo=portefeuille(poids_indice,perf_monte_carlo_actions)
VaR_CVar(perf_indice_monte_carlo)

(-0.02415021635250824,
 performance   -0.029434
 dtype: float64)

In [74]:
print("VaR de l'indice est de " +str(round(VaR_CVar(perf_indice_monte_carlo)[0]*100,3)) + "%")
print("CVaR de l'indice est de " +str(round(VaR_CVar(perf_indice_monte_carlo)[1][0]*100,3)) + "%")

VaR de l'indice est de -2.415%
CVaR de l'indice est de -2.943%


### Relatif

In [75]:
perf_relative=pd.DataFrame()
perf_relative['performance']=portefeuille_monte_carlo['performance']-perf_indice_monte_carlo['performance']

In [76]:
VaR_CVar(perf_relative)

(-0.03825246238529183,
 performance   -0.047563
 dtype: float64)

In [77]:
print("VaR relative est de " +str(round(VaR_CVar(perf_relative)[0]*100,3)) + "%")
print("CVaR relative est de " +str(round(VaR_CVar(perf_relative)[1][0]*100,3)) + "%")

VaR relative est de -3.825%
CVaR relative est de -4.756%


## Vols x1.5

### Portefeuille

In [78]:
monte_carlo=Monte_Carlo(devise_ptf,stress_test,haltons,t)
perf_monte_carlo=performance(monte_carlo,stress_test)
portefeuille_monte_carlo=portefeuille(positions,perf_monte_carlo)

In [79]:
VaR_CVar(portefeuille_monte_carlo)

(-0.15105677945876378,
 performance   -0.166681
 dtype: float64)

In [80]:
print("VaR portefeuille est de " +str(round(VaR_CVar(portefeuille_monte_carlo)[0]*100,3)) + "%")
print("CVaR du portefeuille est de " +str(round(VaR_CVar(portefeuille_monte_carlo)[1][0]*100,3)) + "%")

VaR portefeuille est de -15.106%
CVaR du portefeuille est de -16.668%


### Indice

In [81]:
monte_carlo_indice=Monte_Carlo(devise_indice,stress_test,haltons,t)
perf_monte_carlo_actions=performance(monte_carlo_indice,stress_test)
perf_indice_monte_carlo=portefeuille(poids_indice,perf_monte_carlo_actions)
VaR_CVar(perf_indice_monte_carlo)

(-0.2453670554689817,
 performance   -0.253292
 dtype: float64)

In [82]:
print("VaR de l'indice est de " +str(round(VaR_CVar(perf_indice_monte_carlo)[0]*100,3)) + "%")
print("CVaR de l'indice est de " +str(round(VaR_CVar(perf_indice_monte_carlo)[1][0]*100,3)) + "%")

VaR de l'indice est de -24.537%
CVaR de l'indice est de -25.329%


### Relatif

In [83]:
perf_relative=pd.DataFrame()
perf_relative['performance']=portefeuille_monte_carlo['performance']-perf_indice_monte_carlo['performance']
VaR_CVar(perf_relative)

(0.0713733601800594,
 performance    0.057407
 dtype: float64)

In [84]:
print("VaR relative est de " +str(round(VaR_CVar(perf_relative)[0]*100,3)) + "%")
print("CVaR relative est de " +str(round(VaR_CVar(perf_relative)[1][0]*100,3)) + "%")

VaR relative est de 7.137%
CVaR relative est de 5.741%


# 6) Réduction du risque relatif

In [85]:
assets=dict(zip(portfolio['code'],portfolio['poids']))
spot['EUREUR']=1

In [86]:

#Creation d'un DF qui recence les actifs (en index) et reprends leurs volatilités de la table des facteurs
contribution=pd.DataFrame(assets.items(),columns=['code','poids']).set_index('code')
contribution=contribution.join(facteurs['vol'])
# L'EUR n'a pas de volatilité
contribution.loc['EUREUR','vol']=0
contribution=contribution.reset_index()
contribution['contribution']=0

In [87]:
#Calcul de la variance total du ptf
varcovar=np.array(np.log(1+spot[contribution['code']].pct_change()).cov()*252)
poids=np.array(contribution['poids'])
volatilité=np.array(contribution['vol'])

# Calcul de la variance marginale de chaque actifs dans le portefeuille
for i in range(25):
    contribution.iloc[i,3]=marginal_variance(poids,volatilité,varcovar,i)

In [88]:
# Proportion des variances marginales par rapport à la variance totale (ou somme des variances marginales)
contribution['proportion']=contribution['contribution']/contribution['contribution'].sum()*100

In [89]:
variance(poids,varcovar) , contribution['contribution'].sum()

(0.043085710640333216, 0.04306514500355343)

# a)

In [90]:
# Tri des contributions à la variance
contribution[['code','proportion']].sort_values('proportion',ascending=False)[:20]

,code,proportion
4,URW NA,28.697999
3,MC FP,8.200707
6,FP FP,7.769074
8,BP/ LN,7.398430
5,HSBA LN,6.358015
1,SAP GY,5.210215
9,LIN GY,4.068633
0,ASML NA,3.902126
12,RDSA NA,3.812413
10,SIE GY,3.630402


In [91]:
contributeurs=contribution['proportion'].sort_values(ascending=False)[:20].sum(axis=0)
contributeurs

99.76968626259806

## b)

In [92]:
# Poids des actifs de l'indice
poids_indice=dict(zip(indice_composition['code'],indice_composition['poids']/indice_composition['poids'].sum()))

In [93]:
# Creation d'un set qui reprends tout les actifs du ptf et de l'indice (set = pas de doublons)
all_items=list(indice_composition['code'])
all_items.extend(portfolio['code'])
all_items=set(all_items)

In [94]:
# Creation de deux sets , qui reprends respectivement les actifs du ptf et de l'indice
sets_assets_portfolio=set(positions.keys())
sets_assets_indice=set(poids_indice.keys())

In [95]:
# Prise en comptes des actifs en communs entre l'indice et le ptf
intersection=sets_assets_portfolio.intersection(sets_assets_indice)

In [96]:
# dictionnaire des poids du ptf de différences en fonction de la présence ou non dans l'un des sets
portefeuille_differences={}
for asset in all_items:
    if asset in intersection:
        portefeuille_differences[asset]=positions[asset]-poids_indice[asset]
        
    else:
        if asset in sets_assets_indice:
            portefeuille_differences[asset]=-poids_indice[asset]
        else:
            portefeuille_differences[asset]=positions[asset]

In [97]:
# Convertit les devise GBp, CHF en GBpEUR et CHFEUR...

devise_ptf_diff={}

for asset in all_items:

    if asset in sets_assets_indice:
         devise_ptf_diff[asset]=devise_indice[asset]
            
    else:
          devise_ptf_diff[asset]=devise_ptf[asset]

## c)

In [98]:
monte_carlo_diff=Monte_Carlo(devise_ptf_diff,facteurs,epsilons,t)
perf_monte_carlo_diff=performance(monte_carlo_diff,stress_test)

In [99]:
portefeuille_diff=portefeuille(portefeuille_differences,perf_monte_carlo_diff)

In [100]:
VaR_CVar(portefeuille_diff)

(-0.049565803087995956,
 performance   -0.060617
 dtype: float64)

In [101]:
print("VaR relative est de " +str(round(VaR_CVar(portefeuille_diff)[0]*100,3)) + "%")
print("CVaR relative est de " +str(round(VaR_CVar(portefeuille_diff)[1][0]*100,3)) + "%")

VaR relative est de -4.957%
CVaR relative est de -6.062%


In [102]:
poids_diff=[]

for col in perf_monte_carlo_diff.columns:
    poids_diff.append(portefeuille_differences[col])
    
poids_diff=np.array(poids_diff)

In [103]:
reduction_var=pd.DataFrame()
reduction_var=poids_diff*perf_monte_carlo_diff
reduction_var['performance']=portefeuille_diff

In [104]:
contrib_cvar=reduction_var[reduction_var['performance']<VaR_CVar(reduction_var['performance'])[0]].mean()

In [105]:
contrib_cvar=contrib_cvar/contrib_cvar['performance']*100

In [106]:
contrib_cvar.sort_values()

DG FP           -2.351706
BAS GY          -2.307444
SAF FP          -2.233568
PRU LN          -1.944071
CS FP           -1.748923
ABI BB          -1.737836
UBSG SW         -1.546787
DGE LN          -1.515455
DAI GY          -1.388136
BHP LN          -1.376507
ISP IM          -1.193566
BAYN GY         -1.183988
VOD LN          -1.107282
ZURN SW         -1.093630
ULVR LN         -0.785280
AZN LN          -0.780017
BATS LN         -0.708250
ABBN SW         -0.612610
KER FP          -0.600452
NESN SW         -0.431521
NOVN SW         -0.396321
DTE GY          -0.331967
ADS GY          -0.302289
GSK LN          -0.242605
ENEL IM         -0.211682
CHFEUR          -0.177852
SU FP           -0.147393
REL LN          -0.036495
OR FP           -0.023228
EUREUR          -0.000000
DKKEUR          -0.000000
AI FP            0.006825
ASML NA          0.103388
NG/ LN           0.298995
GBpEUR           0.375452
SAP GY           0.425273
IBE SQ           0.553703
SAN FP           0.613514
LIN GY      

## d)

### Newton Raphson

In [107]:
def value_at_risk(portefeuille,weights): 
    # Fonction qui renvoie la Value At Risk d'un portefeuille
    weighted_portfolio=np.sum(portefeuille*weights,axis=1)
    
    return np.percentile(weighted_portfolio,5)

In [108]:
selected_columns=[]
# On selectionne toutes les colonnes sauf le futur car son poids ne doit pas bouger
for col in perf_monte_carlo_diff.columns:
    if col !='SX5E':
        selected_columns.append(col)
    else:
        continue

In [109]:
portefeuille_à_reduire=perf_monte_carlo_diff[selected_columns]

In [110]:
max_iterations = 25
tolerance = 0.01
#on cree un portefeuille équipondéré qui servira de premier pas ou pas initial
weights = np.array([1/len(selected_columns)]*len(selected_columns))
iteration = 0

In [111]:
while iteration < max_iterations:
    
    var=value_at_risk(portefeuille_à_reduire,weights)

    if var > -0.025:
        break
        
    #On cree un vecteur de gradients pour ajuster les poids en fonction de chaque actifs
    gradient = np.zeros(len(selected_columns))

    for i in range(len(selected_columns)):
        perturbed_weights = weights.copy()
        perturbed_weights[i] += tolerance

        perturbed_var = value_at_risk(portefeuille_à_reduire,perturbed_weights)
        # Derivée de la VaR par rapport au poids
        gradient[i] = (perturbed_var - var) / tolerance

    weights += gradient
    weights = np.clip(weights, 0, 1)
    # On normalise les poids 
    weights /= np.sum(weights)
    

    iteration += 1

In [112]:
# Creation d'un dictionnaire qui comporte les nouveaux poids
dico_nouveau_poids={}
for i in range(len(selected_columns)):
    dico_nouveau_poids[selected_columns[i]]=weights[i]

dico_nouveau_poids['SX5E']=portefeuille_differences['SX5E']

In [113]:
portefeuille_nouveau_poids=portefeuille(dico_nouveau_poids,perf_monte_carlo_diff)

In [114]:
VaR_CVar(portefeuille_nouveau_poids)

(-0.022025648379879498,
 performance   -0.027502
 dtype: float64)

In [115]:
print("VaR portefeuille rebalancé est de " +str(round(VaR_CVar(portefeuille_nouveau_poids)[0]*100,3)) + "%")
print("CVaR portefeuille rebalancé est de " +str(round(VaR_CVar(portefeuille_nouveau_poids)[1][0]*100,3)) + "%")

VaR portefeuille rebalancé est de -2.203%
CVaR portefeuille rebalancé est de -2.75%


### Réduction manuelle

In [116]:
ajustement=contribution.sort_values('proportion',ascending=False).copy()

In [117]:
ajustement=ajustement.loc[ajustement['code'] != 'SX5E', ['code', 'poids','vol','contribution','proportion']]

In [118]:
poids_pre_trade=contribution['poids'].sum()

In [119]:
ajustement

,code,poids,vol,contribution,proportion
4,URW NA,0.080000,0.753004,0.012359,28.697999
3,MC FP,0.060000,0.267885,0.003532,8.200707
6,FP FP,0.040000,0.365865,0.003346,7.769074
8,BP/ LN,0.040000,0.450834,0.003186,7.398430
5,HSBA LN,0.040000,0.378551,0.002738,6.358015
1,SAP GY,0.040000,0.419425,0.002244,5.210215
9,LIN GY,0.050000,0.261399,0.001752,4.068633
0,ASML NA,0.050000,0.30851,0.001680,3.902126
12,RDSA NA,0.030000,0.404642,0.001642,3.812413
10,SIE GY,0.040000,0.304995,0.001563,3.630402


### Ajustement de poids de 7% de URW NA vers CHFEUR

In [120]:
# transfert de poids dans l'un des actifs les plus décorrelés à URW NA
ajustement.loc[4,'poids']=0.01
ajustement.loc[21,'poids']=0.09

### Ajustement de poids de 5% de MC FP vers AZN LN

In [121]:
# transfert de poids dans l'un des actifs les plus décorrelés à MC FP
ajustement.loc[3,'poids']=0.01
ajustement.loc[11,'poids']=0.09

### Ajustement de poids de 3% de BP /LN vers NESN SW

In [122]:
# transfert de poids dans l'un des actifs les plus décorrelés à NESN SW
ajustement.loc[8,'poids']=0.01
ajustement.loc[2,'poids']=0.11

In [123]:
# On met les poids dans un dictionnaire et on ajoute le poids du future
poids_ajuste=dict(zip(ajustement['code'],ajustement['poids']))
poids_ajuste['SX5E']=portefeuille_differences['SX5E']

In [124]:
# On recalcule les vecteurs de variance covariance, de poids et de volatilité
varcovar=np.array(np.log(1+spot[ajustement['code']].pct_change()).cov()*252)
poids=np.array(ajustement['poids'])
volatilité=np.array(ajustement['vol'])

# Calcul de la variance marginale de chaque actifs dans le portefeuille
for i in range(24):
    ajustement.iloc[i,3]=marginal_variance(poids,volatilité,varcovar,i)

In [125]:
# Proportion de variance expliquée par un actif
ajustement['proportion']=ajustement['contribution']/ajustement['contribution'].sum()*100

In [126]:
# On recalcule le portefeuille des différences pour obtenir la VaR relative
portefeuille_differences={}
for asset in all_items:
    if asset in intersection:
        portefeuille_differences[asset]=poids_ajuste[asset]-poids_indice[asset]
        
    else:
        if asset in sets_assets_indice:
            portefeuille_differences[asset]=-poids_indice[asset]
        else:
            portefeuille_differences[asset]=poids_ajuste[asset]

In [127]:
# On refait le monte carlo avec les nouveaux poids
monte_carlo_diff=Monte_Carlo(devise_ptf_diff,facteurs,epsilons,t)
perf_monte_carlo_diff=performance(monte_carlo_diff,stress_test)

In [128]:
portefeuille_differences_ajusté=portefeuille(portefeuille_differences,perf_monte_carlo_diff)

In [129]:
VaR_CVar(portefeuille_differences_ajusté)

(-0.021656553460566753,
 performance   -0.027019
 dtype: float64)

In [130]:
print("VaR relative ajusté est de " +str(round(VaR_CVar(portefeuille_differences_ajusté)[0]*100,3)) + "%")
print("CVaR relative ajusté est de " +str(round(VaR_CVar(portefeuille_differences_ajusté)[1][0]*100,3)) + "%")

VaR relative ajusté est de -2.166%
CVaR relative ajusté est de -2.702%


In [131]:
ajustement

,code,poids,vol,contribution,proportion
4,URW NA,0.010000,0.753004,0.000952,4.270699
3,MC FP,0.010000,0.267885,0.000500,2.241782
6,FP FP,0.040000,0.365865,0.003010,13.508771
8,BP/ LN,0.010000,0.450834,0.000770,3.456339
5,HSBA LN,0.040000,0.378551,0.002047,9.184524
1,SAP GY,0.040000,0.419425,0.001787,8.019065
9,LIN GY,0.050000,0.261399,0.001877,8.422036
0,ASML NA,0.050000,0.30851,0.001144,5.132072
12,RDSA NA,0.030000,0.404642,0.001579,7.084114
10,SIE GY,0.040000,0.304995,0.001385,6.216578


In [132]:
# Verification des poids
poids_post_trade=np.array(list(poids_ajuste.values())).sum()
poids_post_trade==poids_pre_trade

True